In [ ]:
import pandas as pd
from currency_converter import CurrencyConverter

from constants import DATAFRAMES

# Data Cleaning

The goal is to fix or handle data quality issues identified in the validation stage.

NOTE: From data cleaning stage onwards, I will work with `clients.csv`, `trader.csv` and `trans.csv`. The `a2p_ref.csv` and `partner_codes.csv` are more suitable for partner attrition analysis which is not part of the assignment.

In [ ]:
display(f'Initial record counts:')
for name, df in DATAFRAMES.items():
    display(f'{name}: {len(df)}')

## 1. Filter Invalid Transaction Amounts

In [ ]:
invalid_amounts = DATAFRAMES['df_trans'][DATAFRAMES['df_trans'].amount <= 0]
display(f"Found {len(invalid_amounts)} transactions with amount <= 0")
display(invalid_amounts.head())

In [ ]:
df_trans_clean = DATAFRAMES['df_trans'][DATAFRAMES['df_trans'].amount > 0].copy()
display(f"Removed {len(DATAFRAMES['df_trans']) - len(df_trans_clean)} transactions with invalid amounts")
display(f"Remaining transactions: {len(df_trans_clean)}")

## 2. Filter Invalid Registration Types

In [ ]:
display("Registration types found:")
display(DATAFRAMES['df_clients'].type.value_counts())

In [ ]:
valid_types = ['Full', 'Light', 'Demo']
invalid_types = DATAFRAMES['df_clients'][~DATAFRAMES['df_clients'].type.isin(valid_types)]
display(f"Found {len(invalid_types)} clients with invalid registration types")

In [ ]:
df_clients_clean = DATAFRAMES['df_clients'][DATAFRAMES['df_clients'].type.isin(valid_types)].copy()
display(f"Removed {len(DATAFRAMES['df_clients']) - len(df_clients_clean)} clients with invalid registration types")
display(f"Remaining clients: {len(df_clients_clean)}")

## 3. Remove Orphaned Records - Referential Integrity

### 3.1 Remove traders with non-existent clients

In [ ]:
orphaned_traders = DATAFRAMES['df_trader'][~DATAFRAMES['df_trader'].client.isin(df_clients_clean.client)]
display(f"Found {len(orphaned_traders)} trader records with non-existent clients")
display(f"Percentage: {len(orphaned_traders) / len(DATAFRAMES['df_trader']) * 100:.2f}%")

In [ ]:
df_trader_clean = DATAFRAMES['df_trader'][DATAFRAMES['df_trader'].client.isin(df_clients_clean.client)].copy()
display(f"Removed {len(DATAFRAMES['df_trader']) - len(df_trader_clean)} orphaned trader records")
display(f"Remaining traders: {len(df_trader_clean)}")

### 3.2 Remove transactions with non-existent logins

In [ ]:
orphaned_trans = df_trans_clean[~df_trans_clean.login.isin(df_trader_clean.login)]
display(f"Found {len(orphaned_trans)} transactions with non-existent logins")
display(f"Percentage: {len(orphaned_trans) / len(df_trans_clean) * 100:.2f}%")

In [ ]:
df_trans_clean = df_trans_clean[df_trans_clean.login.isin(df_trader_clean.login)].copy()
display(f"Removed {len(orphaned_trans)} orphaned transaction records")
display(f"Remaining transactions: {len(df_trans_clean)}")

### 3.3 Verify first_deposit_id references

In [ ]:
invalid_first_deposits = df_trader_clean[df_trader_clean.first_deposit_id.notna() & ~df_trader_clean.first_deposit_id.isin(df_trans_clean.transaction_id)]
display(f"Found {len(invalid_first_deposits)} traders with invalid first_deposit_id references")


In [ ]:
if len(invalid_first_deposits) > 0:
    display("Setting invalid first_deposit_id to null")
    df_trader_clean.loc[
        (invalid_first_deposits.login.values == df_trader_clean.login.values) &
        (invalid_first_deposits.client.values == df_trader_clean.client.values) &
        (invalid_first_deposits.account_id.values == df_trader_clean.account_id.values), 'first_deposit_id'] = None
    display(f"Set {len(invalid_first_deposits)} first_deposit_id values to null")

## 4. Data Type Conversions

In [ ]:
id_columns = ['account_id', 'first_deposit_id']
for col in id_columns:
    df_trader_clean[col] = df_trader_clean[col].astype('Int64')

In [ ]:
display(df_trader_clean.info())
display(df_trader_clean.head())

In [ ]:
df_trans_clean['amount'] = df_trans_clean['amount'].astype('Float64')

In [ ]:
display(df_trans_clean.info())
display(df_trans_clean.head())

## 5. Handle Missing Values

In [ ]:
display("Missing values in df_trans_clean:")
display(df_trans_clean.isnull().sum())

display("\nMissing values in df_trader_clean:")
display(df_trader_clean.isnull().sum())

display("\nMissing values in df_clients_clean:")
display(df_clients_clean.isnull().sum())

NOTE: account_id and first_deposit_id can be null - this is acceptable. 
 - the trader registered but never opened an account
 - the trader has an account but hasn't made a deposit yet

## 6. Handle currency conversion

In [ ]:
c = CurrencyConverter(fallback_on_missing_rate=True)
df_trans_clean['amount_USD'] = df_trans_clean.apply(
    lambda x: c.convert(x['amount'], x['currency'], 'USD', date=x['created_at'].date()), axis=1
)

In [ ]:
display(df_trans_clean.info())
display(df_trans_clean.head())

## 7. Verify Cleaned Data

In [ ]:
display("Verifying referential integrity after cleaning:")
display(f"All traders reference valid clients: {set(df_trader_clean.client).issubset(set(df_clients_clean.client))}")
display(f"All transactions reference valid logins: {set(df_trans_clean.login).issubset(set(df_trader_clean.login))}")
display(f"All first_deposit_ids reference valid transactions: {set(df_trader_clean.first_deposit_id.dropna()).issubset(set(df_trans_clean.transaction_id))}")

In [ ]:
display("Verifying logical constraints:")
display(f"All amounts > 0: {all(df_trans_clean.amount > 0)}")
display(f"All registration types valid: {set(df_clients_clean.type.unique()).issubset({'Full', 'Light', 'Demo'})}")

## 8. Summary Statistics

In [ ]:
display("Cleaning Summary")
display(f"Clients: {len(DATAFRAMES['df_clients'])} → {len(df_clients_clean)} (removed {len(DATAFRAMES['df_clients']) - len(df_clients_clean)})")
display(f"Traders: {len(DATAFRAMES['df_trader'])} → {len(df_trader_clean)} (removed {len(DATAFRAMES['df_trader']) - len(df_trader_clean)})")
display(f"Transactions: {len(DATAFRAMES['df_trans'])} → {len(df_trans_clean)} (removed {len(DATAFRAMES['df_trans']) - len(df_trans_clean)})")

## 9. Save Cleaned Data

In [ ]:
path = '../../data/processed/'

df_clients_clean.to_csv(path + 'clients.csv', index=False, sep=';')
df_trader_clean.to_csv(path + 'trader.csv', index=False, sep=';')
df_trans_clean.to_csv(path + 'trans.csv', index=False, sep=';')